#### Comparison of Rxn alerts with RFC predictions for TSCA set

Prepared by: Grace Patlewicz <br>
Last modified: 30 January 2023 <br>
Changes: Comparing predictions from the alerts used in Nelms et al (2019) with the predictions made by the RFC for the TSCA set

In [1]:
import os


In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
import openpyxl

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
processed_dir = TOP + 'data/processed/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'


In [6]:
df = pd.read_csv(external_dir+'rxn_domains.csv')

In [7]:
df[df['Mech_Domain'] == 'MA']

,#,Mech_Domain,Alert,SMARTS_pattern
11,2,MA,Michael Acceptor,[CX3!H0]=[#6]-[#6]=O
12,2,MA,Michael Acceptor,[$([#7]=[#6]-1-[#6]=[#6!H0]-[#6](=[#7])-[#6]=[...
13,2,MA,Michael Acceptor,[C!H0]=[C]-[#6]=O
14,2,MA,Michael Acceptor,[CX3!H0]=[#6]C#N
15,2,MA,Michael Acceptor,[CX3!H0]=[#6]-[NX3](=O)(=O)
16,2,MA,Michael Acceptor,[#8-]-[#7+](=O)-[#6]=[CX3!H0]
17,2,MA,Michael Acceptor,[CX3!H0]=[#6]-[#6]-[NX3+](=O)[O-]
18,2,MA,Michael Acceptor,"[$(O=[#6]-1-[#6!H0]=[#6]-1),$(O=[#6]-1-[#6]=[#..."
19,2,MA,Michael Acceptor,"[$([#6;A][#6,O]-[#6](=O)C#[#6!H0]),$(O=[#6]C#[..."
20,2,MA,Michael Acceptor,[$([#6!H0]=[#6]-[#6]-1=[#6]-[#6]=[#6]-[#6]=[#7...


In [8]:
df = df.set_index('#')

In [9]:
def grouper(index):
    floatindex = float(index)
    intindex = int(floatindex)        
    return intindex

In [10]:
smarts_df = df.groupby(grouper)

In [11]:
mech_dict = {}
for i,group in smarts_df:
    key = [name for name in group['Mech_Domain'] if name!=' '][0]
    value = set(group['SMARTS_pattern'])
    mech_dict[key]=value

In [12]:
Dow_smarts = {k : [Chem.MolFromSmarts(s) for s in v] for k,v in mech_dict.items()}

In [14]:
#RDkit smarts for the set of reactivity alerts published by Dow. Dictionary of the smarts patterns
#Dow_smarts['MA']

In [15]:
enoch = pd.read_csv(external_dir+'Enoch.csv')

In [16]:
enoch.head()

,#,Mech,SMARTS
0,1,SNAR,"c1([F,Cl,Br,I,$(N(=O)~O)])c([F,Cl,Br,I,$(N(=O)..."
1,1,SNAR,"c1([F,Cl,Br,I,$(N(=O)~O)])c([F,Cl,Br,I,$(N(=O)..."
2,1,SNAR,"c1([F,Cl,Br,I,$(N(=O)~O)])ncc([F,Cl,Br,I,$(N(=..."
3,1,SNAR,"c1([F,Cl,Br,I,$(N(=O)~O)])ncccc1([F,Cl,Br,I,$(..."
4,1,SNAR,"c1([F,Cl,Br,I,$(N(=O)~O)])ncccn1"


In [17]:
enoch_df = enoch.set_index('#')

In [18]:
enoch_df = enoch_df.groupby(grouper)

In [19]:
enoch_dict = {}
for i,group in enoch_df:
    key = [name for name in group['Mech'] if name!=' '][0]
    value = set(group['SMARTS'])
    enoch_dict[key]=value

In [20]:
enoch_smarts = {k : [Chem.MolFromSmarts(s) for s in v] for k,v in enoch_dict.items()}

In [21]:
{k:len(v) for k,v in enoch_dict.items()}

{'SNAR': 8, 'SB': 14, 'MA': 26, 'Acyl': 9, 'SN2': 9}

In [22]:
{k:len(v) for k,v in mech_dict.items()}

{'Acyl': 11, 'MA': 19, 'SNAR': 14, 'SB': 4, 'SN2': 23}

In [23]:
enoch_crt = pd.read_csv(external_dir+'Enoch_CRT.csv')

In [24]:
enoch_crt_df = enoch_crt.set_index('#')

In [25]:
enoch_crt = enoch_crt_df.groupby(grouper)

In [26]:
crt_dict = {}
for i,group in enoch_crt:
    key = [name for name in group['Mech'] if name!=' '][0]
    value = set(group['SMARTS'])
    crt_dict[key]=value

In [27]:
oecd_smarts = {k : [Chem.MolFromSmarts(s) for s in v] for k,v in crt_dict.items()}

In [28]:
crt_len = {k:len(v) for k,v in crt_dict.items()}

In [29]:
crt_len

{'Acyl': 17, 'MA': 34, 'SB': 11, 'SN2': 39, 'SNAr': 1}

In [30]:
enoch_len = {k:len(v) for k,v in enoch_dict.items()}
dow_len = {k:len(v) for k,v in mech_dict.items()}

In [31]:
print(sum([e for e in crt_len.values()]))

print(sum([e for e in enoch_len.values()]))
print(sum([e for e in dow_len.values()]))

102
66
71


In [34]:
tsca_smi = pd.read_csv(raw_dir+'Chemical List TSCA_ACTIVE_NCTI_0320-2022-06-02.txt', sep = '\t', header = None)

In [36]:
tsca_smi.columns = ['smiles', 'dtxsid']

In [37]:
len(tsca_smi)

14365

In [48]:
tsca_smi = tsca_smi[tsca_smi['smiles'].notnull()]

In [49]:
#Dictionary of the jrc chemicals
tsca_dict = { k:v for (k,v) in zip(tsca_smi['dtxsid'], tsca_smi['smiles'])}

In [51]:
#tsca_dict

In [52]:
#jrc rdkit dictionary
tsca_smiles = {k : Chem.MolFromSmiles(v) for k,v in tsca_dict.items() }

RDKit ERROR: [08:49:14] Explicit valence for atom # 1 C, 6, is greater than permitted
RDKit ERROR: [08:49:14] Explicit valence for atom # 2 O, 3, is greater than permitted
RDKit ERROR: [08:49:14] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
RDKit ERROR: 
RDKit ERROR: [08:49:14] Explicit valence for atom # 8 O, 3, is greater than permitted
RDKit ERROR: [08:49:14] Explicit valence for atom # 0 O, 3, is greater than permitted
RDKit ERROR: [08:49:14] Explicit valence for atom # 2 O, 3, is greater than permitted
RDKit ERROR: [08:49:14] Explicit valence for atom # 1 Cl, 7, is greater than permitted
RDKit ERROR: [08:49:14] Explicit valence for atom # 2 O, 3, is greater than permitted
RDKit ERROR: [08:49:14] Explicit valence for atom # 4 C, 5, is greater than permitted
RDKit ERROR: [08:49:14] Explicit valence for atom # 3 C, 5, is greater than permitted
RDKit ERROR: [08:49:14] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5
RDKit ERROR: 
RDKit ERROR: [08:49:14] Explicit valence for atom

In [57]:
tsca_smiles = {k:v for k,v in tsca_smiles.items() if v is not None}

In [58]:
tsca_Dow = {x:[k for k, v in Dow_smarts.items() if any([y.HasSubstructMatch(e) for e in v])] for x,y in tsca_smiles.items()}

In [59]:
tsca_Enoch = {x:[k for k, v in enoch_smarts.items() if any([y.HasSubstructMatch(e) for e in v])] for x,y in tsca_smiles.items()}

In [60]:
tsca_OECD = {x:[k for k, v in oecd_smarts.items() if any([y.HasSubstructMatch(e) for e in v])] for x,y in tsca_smiles.items()}

In [64]:
oecd_df = pd.DataFrame(list(tsca_OECD.items()))

In [65]:
enoch_df = pd.DataFrame(list(tsca_Enoch.items()))

In [66]:
dow_df = pd.DataFrame(list(tsca_Dow.items()))

In [67]:
dow_df.shape

(14272, 2)

In [69]:
enoch_df.shape

(14272, 2)

In [70]:
oecd_df.shape

(14272, 2)

In [72]:
dow_df.columns = ['dtxsid', 'dow_alerts']

In [73]:
enoch_df.columns = ['dtxsid', 'enoch_alerts']
oecd_df.columns = ['dtxsid', 'oecd_alerts']

In [74]:
dow_df =dow_df.set_index('dtxsid')
enoch_df = enoch_df.set_index('dtxsid')
oecd_df = oecd_df.set_index('dtxsid')

all_df = pd.concat([dow_df, enoch_df, oecd_df], axis = 1)

In [80]:
def convert_df(s):
    return s.apply(lambda x: 0 if len(x) == 0 else 1)

In [88]:
for col in all_df.columns:
    all_df[col] = convert_df(all_df[col])

In [90]:
all_df
all_df['OverAll'] = all_df.sum(1)

In [92]:
all_df['final'] = all_df['OverAll'].apply(lambda x: 1 if x >=1 else 0)
    

In [94]:
all_df['final'].value_counts()

0    8605
1    5667
Name: final, dtype: int64

In [95]:
all_df.to_csv(external_dir+'nelms_tsca_300123.csv')